In [ ]:
import pandas as pd
import time
import google.generativeai as genai

gemini_question_df = pd.read_csv("../Evaluation-Questions.csv")

In [ ]:
# API Key
API_KEY_GOOGLE='API_KEY_GOOGLE'
genai.configure(api_key=API_KEY_GOOGLE)

# Gemini 1.5 No Contxt

In [ ]:
answers_gemini_1_5 = []

In [ ]:
# Define number of questions to process in this iteration
num_questions_to_process = 35

# Calculate start and end indices based on current loop position
start_index = min(len(answers_gemini_1_5), len(gemini_question_df['Question']))  # Start from where answers left off
end_index = min(start_index + num_questions_to_process, len(gemini_question_df['Question']))


print(start_index)
print(end_index)

In [ ]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

convo = model.start_chat(history=[])

# iterate through the questions
for i in range(start_index, end_index):
    question_prompt = gemini_question_df['Question'][i]
    print(gemini_question_df['Question'][i])

    try:
        convo.send_message(question_prompt)
    except Exception as e:
        time.sleep(120)
        convo.send_message(question_prompt)

    gemini_question_df.loc[i, 'LLM'] = 'gemini-1.5-pro-latest'

    response = convo.last.text
    answers_gemini_1_5.append(str(response))
    print(response)

In [ ]:
# update Dataframe
gemini_question_df['answer'] = pd.DataFrame({'answers': answers_gemini_1_5})

In [ ]:
filename = './csv-NoContext/Gemini-1-5_EvaluationQuestions.csv'
gemini_question_df.to_csv(filename, sep=',', index=False, encoding='utf-8')